In [ ]:
import cv2, pytesseract, re, json, sys, requests
from flask import Flask,render_template,url_for,request
import re
import pandas as pd
import numpy as np
import spacy
from spacy import displacy
import en_core_web_sm
nlp = spacy.load('en_core_web_sm')
from PIL import Image

In [ ]:
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract'
TESSDATA_PREFIX = 'C:/Program Files/Tesseract-O'

In [ ]:
def ocr_core(file):
    #img = cv2.imdecode(np.fromstring(request.files['file'].read(), np.uint8), cv2.IMREAD_UNCHANGED)
    img = cv2.imread(file)
    img=cv2.resize(img,None,fx=0.4,fy=0.4)
    img1 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img2 = cv2.adaptiveThreshold(img1, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 65, 2)
    config="--psm 3"
    OCR_extraction = pytesseract.image_to_string(img)
    return OCR_extraction

In [ ]:
print(ocr_core('Pan.jpg'))

In [ ]:
# define a folder to store and later serve the images
UPLOAD_FOLDER = '/static/uploads/'
# allow files of a specific type
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg'])

In [ ]:
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
# function to check the file extension
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

In [ ]:
'''@app.route('/')
def upload():
    return render_template("index.html")'''

In [ ]:
# route and function to handle the upload page
@app.route('/', methods=['GET', 'POST'])
def uploa():
    global file,file1
    #file=request.file
    file1=request.files
    if request.method == 'POST':
        # check if there is a file in the request
        if 'file' not in request.files:
            return render_template('upload.html', msg='No file selected')
            #file = request.files['file']
        # if no file is selected
       # if file.filename == '':
           # return render_template('upload.html', msg='No file selected')

        if file1['file'] and allowed_file(file1['file'].filename):
            OCR_extraction = ocr_core(file1)

            return render_template('upload.html',
                                   msg='Successfully processed',
                                   extracted_text=OCR_extraction,
                                   img_src=UPLOAD_FOLDER + file.filename)
    elif request.method == 'GET':
        return render_template('upload.html')

In [ ]:
file1['file'].filename

In [ ]:
if __name__ == '__main__':
    app.run()

In [ ]:
@app.route('/process',methods=["POST"])
def process():
    if request.method == 'POST':
        img = request.form['image']
        img = cv2.imread("img")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.GaussianBlur(img,(5,5),0)
        img = cv2.threshold(img, 100, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY)[1]
        OCR_extraction = pytesseract.image_to_string(img)
    return render_template("index.html",results=OCR_extraction)

In [ ]:
if __name__ == '__main__':
    app.run()